플라스크에 저장된 롤셋을 가져와서 AI팀이 추출한 특징과 매칭해 심리 진단 결과를 도출하는 과정

In [ ]:
import flask
from flask import Flask, jsonify, request, Response
from flask_restful import reqparse
from datetime import datetime
import pymysql
import json
import uuid	

In [ ]:
app = Flask(__name__)
app.config["DEBUG"] = True

config = {
    'host': 'insideout.cpa0spimmjj8.us-east-2.rds.amazonaws.com',
    'port': 3306,
    'user': 'admin',
    'password':'insideout',
    'database': 'insideout'
}


# result 테이블에 데이터 입력
def insert_data(attribute, result):
		try:
				conn = pymysql.connect(**config)
				cur = conn.cursor()
				setdata = (attribute, result)
				cur.execute("INSERT INTO result(attribute, result) VALUES(%s, %s)", setdata)
				conn.commit()
		except pymysql.IntegrityError:
        return Response(dumps({"message": "실패"}), status=200, mimetype='application/json')
    finally:
        conn.close()

# Feature_Analysis 테이블에서 데이터 추출
def select_all():
		ret = list()
		try:
				conn = pymysql.connect(**config)
				cur = conn.cursor()
				cur.execute("SELECT * FROM Feature_Analysis")
				ret = cur.fetchall()
		except pymysql.IntegrityError:
        return Response(dumps({"message": "실패"}), status=200, mimetype='application/json')
    finally:
				conn.close()
				return ret
		

@app.route('/result_insert')
# result 테이블에 분석 결과 데이터 추가
def result_insert(result, answer):  #result와 answer도 받아오는 과정이 필요한가?
		info = select_all()
		
		import pandas as pd
		feature_analysis = pd.DataFrame(info)
  
		ans = []
	  ans.append(answer['A1'])

	  if answer['A2'] < answer['A3']:
		    ans.append(0)
	  else:
		    ans.append(1)

	  att, res = feature_analysis[(feature_analysis['root']==result[0]) & (feature_analysis['branch']==result[1]) & 
                              (feature_analysis['fruit']==result[2]) & (feature_analysis['leaves']==result[3]) &
                              (feature_analysis['knot']==result[4]) & (feature_analysis['A1']==ans[0]) &
                              (feature_analysis['new_A2']==ans[1])][['attribute','result']].iloc[0]
	  att = 'type' + str(att)

		insert_data(att, res)
		
	  return att, res





if __name__ == '__main__':
		select_data()
    app.run(host='0.0.0.0')